# Tensorflow-1.0 in Action 

In [1]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
print("Keras version: {}".format(tf.keras.__version__))

TensorFlow version: 1.15.0
Keras version: 2.2.4-tf


In [2]:
variable = tf.Variable([3, 3])
if tf.test.is_gpu_available():
    print('GPU')
    print('GPU #0?')
    print(var.device.endswith('GPU:0'))
else:
    print('CPU')


CPU


In [9]:
sa = tf.constant('sahil')
sa?

Type:           Tensor
String form:    Tensor("Const_6:0", shape=(), dtype=string)
File:           c:\users\magic\anaconda3\envs\dl2\lib\site-packages\tensorflow_core\python\framework\ops.py
Docstring:     
Represents one of the outputs of an `Operation`.

A `Tensor` is a symbolic handle to one of the outputs of an
`Operation`. It does not hold the values of that operation's output,
but instead provides a means of computing those values in a
TensorFlow `tf.compat.v1.Session`.

This class has two primary purposes:

1. A `Tensor` can be passed as an input to another `Operation`.
   This builds a dataflow connection between operations, which
   enables TensorFlow to execute an entire `Graph` that represents a
   large, multi-step computation.

2. After the graph has been launched in a session, the value of the
   `Tensor` can be computed by passing it to
   `tf.Session.run`.
   `t.eval()` is a shortcut for calling
   `tf.compat.v1.get_default_session().run(t)`.

In the following example, 

In [10]:
type(sa)

tensorflow.python.framework.ops.Tensor

In [21]:
ar = tf.constant([1,2,3])

In [38]:
with tf.Session() as sess:
    # print(sess.list_devices())
    t = sess.run(sa)
    print(type(t))
    t2 = sess.run([ar])
    print(type(t2))
    print(t2)
    t3 = sess.run(ar)
    print(type(t3))
    print(t3)

<class 'bytes'>
<class 'list'>
[array([1, 2, 3])]
<class 'numpy.ndarray'>
[1 2 3]


In [36]:
sess

In [35]:
print(t)

b'sahil'


In [13]:
ses = tf.compat.v1.Session() # new way (in 2.x) of initiating session 

In [15]:
print(ses)

In [32]:
# A is a 0-dimensional int32 tensor
A = tf.constant(1234)

# B is a 1-dimensional int32 tensor
B = tf.constant([123,456,789])

# C is a 2-dimensional int32 tensor
C = tf.constant([ [123,456,789], [222,333,444] ])
A,B,C

(<tf.Tensor 'Const_8:0' shape=() dtype=int32>,
 <tf.Tensor 'Const_9:0' shape=(3,) dtype=int32>,
 <tf.Tensor 'Const_10:0' shape=(2, 3) dtype=int32>)

In [59]:
x = tf.placeholder(tf.string)
sess = tf.Session()
output = sess.run(x, feed_dict={x: 'ok'})
print(output)

ok


In [60]:
sess.run(x,feed_dict={x:2})

TypeError: Type of feed value 2 with type <class 'int'> is not compatible with Tensor type <class 'object'>. Try explicitly setting the type of the feed tensor to a larger type (e.g. int64).

In [61]:
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)

output_xy = sess.run([x,y],feed_dict={x: 'Test String', y: 123, z: 45.67})
# output_xy2 = sess.run({x,y},feed_dict={x: 'Test String', y: 123, z: 45.67}) # this does not work
output_yz = sess.run((y,z), feed_dict={x: 'Test String', y: 123, z:45.67})
output_y = sess.run(y, feed_dict={x: 'Test String', y: 123, z:45.67})
print(output_xy)
# print(output_xy2)
print(output_yz)
print(output_y)

[array('Test String', dtype=object), array(123)]
(array(123), array(45.67, dtype=float32))
123


- ## Mathematics

In [62]:
x = tf.add(5, 2)  # 7
y = tf.subtract(10, 4) # 6
z = tf.multiply(2, 5)  # 10
output = sess.run([x,y,z])
print(output)

[7, 6, 10]


>>>> But we do need to take care of the data types!

In [63]:
tf.subtract(tf.constant(2.0),tf.constant(1))  

TypeError: Input 'y' of 'Sub' Op has type int32 that does not match type float32 of argument 'x'.

>>>> In such cases you might need type casting

In [64]:
c = tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))   # 1
output = sess.run(c)
print(output)

1


In [65]:
x = tf.constant(10)
y = tf.constant(2)
z = tf.subtract(tf.divide(x, y), 1)
output = sess.run(z)
print(output)

4.0


tf.placeholder() and tf.constant(), since those Tensors can't be modified. This is where tf.Variable class comes in.

In [67]:
x = tf.Variable(5)

The tf.Variable class creates a tensor with an initial value that can be modified, much like a normal Python variable. This tensor stores its state in the session, so you must initialize the state of the tensor manually. You'll use the tf.global_variables_initializer() function to initialize the state of all the Variable tensors:

In [68]:
init = tf.global_variables_initializer()
sess.run(init)

choosing weights from a normal distribution prevents any one weight from overwhelming other weights. We'll use the tf.truncated_normal() function to generate random numbers from a normal distribution.

In [69]:
n_features = 120
n_labels = 5
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

weights

<tf.Variable 'Variable_3:0' shape=(120, 5) dtype=float32_ref>

The tf.truncated_normal() function returns a tensor with random values from a normal distribution whose magnitude is no more than 2 standard deviations from the mean.

Since the weights are already helping prevent the model from getting stuck, you don't need to randomize the bias. Let's use the simplest solution, setting the bias to 0.

In [70]:
n_labels = 5
bias = tf.Variable(tf.zeros(n_labels)) # generates list of zeros

The `tf.zeros()` function returns a tensor with all zeros.

## TensorFlow Softmax

The softmax function squashes it's inputs, typically called **logits** or **logit scores**, to be between 0 and 1 and also normalizes the outputs such that they all sum to 1. This means the output of the softmax function is equivalent to a categorical probability distribution. It's the perfect function to use as the output activation for a network predicting multiple classes.

In [71]:
logit_data = [19,354,354,45,354,54]
logits = tf.placeholder(tf.float32)

# Calculate the softmax of the logits
softmax = tf.nn.softmax(logits)   

output = sess.run(softmax, feed_dict={logits: logit_data})
output

array([0.        , 0.33333334, 0.33333334, 0.        , 0.33333334,
       0.        ], dtype=float32)

In [72]:
sum(output) # sum of the output ~ 1

1.0000000298023224

The same sum can be performed by tf

In [73]:
x = tf.reduce_sum(output)  # 15

In [74]:
sess.run(x)

1.0

### Natural Log

In [77]:
sess.run(tf.log(100.0))

4.6051702

### Imitating an single point input softmax

In [79]:
softmax_data = [0.7, 0.2, 0.1]
one_hot_data = [1.0, 0.0, 0.0]

softmax = tf.placeholder(tf.float32)
one_hot = tf.placeholder(tf.float32)

cross_entropy = -tf.reduce_sum(tf.multiply(one_hot, tf.log(softmax)))

output = sess.run(cross_entropy, feed_dict={one_hot: one_hot_data, softmax: softmax_data})
output

0.35667497